#### Ingestion

In [0]:
#configurations

In [0]:
%run "../utils/configs/functions"

In [0]:
%run "../utils/configs/paths"

In [0]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, DoubleType, DateType

In [0]:
#schema
#It involves nested JSON File, So need to define Nested Elements schema separately, which will be inherted by new formed parent schema
name_sch = StructType(fields = [ StructField("forename", StringType(), True),
                                 StructField("surname", StringType(), True)   
    
])
sch = StructType(fields = [StructField("driverId", IntegerType(), False),
                                    StructField("driverRef", StringType(), True),
                                   StructField("number", IntegerType(), True),
                                   StructField("code", StringType(), True),
                                   StructField("name", name_sch, True),
                                   StructField("dob", DateType(), True),
                                   StructField("nationality", StringType(), True) ,
                                   StructField("url", StringType(), True) 
    
])

In [0]:
output = filename("2021-03-21")
df = spark.read.schema(sch).json(f"{raw_path}/{output}/drivers.json")

#### Transformations (Columns)

In [0]:
from pyspark.sql.functions import current_timestamp, lit, col, concat

In [0]:
final_df = df.withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname"))) \
.withColumn("ingestion_date", current_timestamp()) \
.withColumnRenamed("driverId", "driver_id") \
.withColumnRenamed("driverRef", "driver_ref") \
.drop("url")

##### Writing to Spark Tables As Parquet format

In [0]:
final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.drivers")